## ライブラリインポート

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
import tensorflow_datasets as tfds

#モデル定義のクラス
from tensorflow.keras import models
#クラス分類用のクラス．
from tensorflow.keras.utils import to_categorical
#使うlayer達
from tensorflow.keras.layers import Dense, Dropout, Flatten
#tensorflowの提供するVGG-16
from tensorflow.keras.applications.vgg16 import VGG16
#preprocess_input:入力データを最適化するクラス．decode_predictions:出力をデコードするクラス．
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions

from tensorflow.keras.utils import to_categorical

#おまじないコード
#警告文のレベルを下げる
import os 
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

/opt/anaconda3/envs/tensorflow_New/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## GPU設定

In [4]:
#利用可能なGPUが存在するか確認する．
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)
        print('{} memory growth: {}'.format(device, tf.config.experimental.get_memory_growth(device)))
else:
    print("Not enough GPU hardware devices available")

Not enough GPU hardware devices available


## 作成したデータセットの読み込み

In [5]:
learning_dataset = 'sndl_size_discrim'     # version

# 作成したデータセットの読み込み
dataset, dataset_metadata = tfds.load(learning_dataset, 
                                      with_info = True, 
                                      shuffle_files = True,
                                      as_supervised = True,
                                      batch_size = -1)

print(dataset_metadata)

tfds.core.DatasetInfo(
    name='sndl_size_discrim',
    full_name='sndl_size_discrim/1.0.0',
    description="""
    Custom dataset with 'large' and 'small' labels.
    """,
    homepage='https://www.tensorflow.org/datasets/catalog/sndl_size_discrim',
    data_path='/Users/yonekawahiroto/tensorflow_datasets/sndl_size_discrim/1.0.0',
    file_format=tfrecord,
    download_size=Unknown size,
    dataset_size=780.24 KiB,
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=2),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=96, num_shards=1>,
        'train': <SplitInfo num_examples=96, num_shards=1>,
    },
    citation="""""",
)


In [6]:
C = dataset_metadata.features['label'].num_classes #カテゴリー数
Ntrain = dataset_metadata.splits['train'].num_examples #学習画像数
Ntest = dataset_metadata.splits['test'].num_examples #検証画像数
Nbatch = 1 #バッチ数

train = dataset['train']
x_train, y_train = train
test = dataset['test']
x_test, y_test = test
print(C,Ntrain,Ntest)



2 96 96


In [7]:
print(np.shape(train))

/opt/anaconda3/envs/tensorflow_New/lib/python3.8/site-packages/numpy/core/fromnumeric.py:2009: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape


ValueError: could not broadcast input array from shape (96,224,224,3) into shape (96,)

## 転移学習

### モデル定義

In [6]:
input_shape = (224, 224, 3)

vgg16_model = VGG16(weights='imagenet',
                    include_top = False,
                    input_shape=input_shape,
                    classes=1000)

### モデルのパラメータの学習を止める

In [7]:
vgg16_model.trainable = False

In [8]:
vgg16_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

### モデルの構築

In [9]:
# input_top = vgg16_model.output
# x = Flatten(input_shape=vgg16_model.output_shape[1:], name='flatten')(input_top)
# x = Dense(4096,activation='relu', name='fc1')(x)
# x = Dropout(0.5, name='do1')(x)
# x = Dense(4096,activation='relu', name='fc2')(x)
# x = Dropout(0.5, name='do2')(x)
# prediction = Dense(2,activation='softmax', name='prediction')(x)
    
# actmodel = models.Model(inputs=vgg16_model.input, outputs=prediction)


In [10]:
# actmodel.summary()

In [11]:

def Create_model(input_shape, C):
    model = models.Sequential()
    model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(input_shape)))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dense(2))

    return model

actmodel = Create_model((224, 224, 3), 1)

### モデルコンパイル

In [12]:
base_learning_rate = 0.0001
actmodel.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

### データセットジェネレータ作成

In [13]:
def dateset_generator(dataset, batch_size=None, num_classes=C):
    images = np.zeros((batch_size,input_shape[0],input_shape[1],input_shape[2]))
    labels = np.zeros((batch_size, num_classes))
    while True:
        count = 0
        dataset_numpy = tfds.as_numpy(dataset)
        for image, label in zip(dataset_numpy[0],dataset_numpy[1]):
            
            re_image = np.round(image).astype(np.int64)
            # re_image = tf.image.random_crop(re_image,size=(input_shape[0],input_shape[1],input_shape[2])) 
            
            
            images[count%batch_size] = np.expand_dims(preprocess_input(re_image),0)
            labels[count%batch_size] = np.expand_dims(to_categorical(label, num_classes=num_classes),0)

            count += 1
            if(count%batch_size == 0):
                yield images, labels

In [15]:
hi = actmodel.fit(dateset_generator(train, Nbatch),
                  dateset_generator(test, Nbatch))

ValueError: `y` argument is not supported when using python generator as input.

### 現時点の精度確認

In [14]:
Nbatch = 4
loss0, accuracy0 = actmodel.evaluate(dateset_generator(test, Nbatch), steps=Ntest/Nbatch)

24/24 [==============================] - 1s 39ms/step - loss: 4.1167 - accuracy: 0.5000


In [ ]:
print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

initial loss: 8.85
initial accuracy: 0.12
